In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [2]:
import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(-532.6914, shape=(), dtype=float32)


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7116849697585654385
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4118216704
locality {
  bus_id: 1
  links {
  }
}
incarnation: 231886293659270475
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


In [4]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
print(tf.__version__)

2.9.1


In [7]:
tf.test.is_built_with_cuda()

True

In [9]:
# https://www.analyticsvidhya.com/blog/2021/11/benchmarking-cpu-and-gpu-performance-with-tensorflow/

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-w18bky_9 because the default path (/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [10]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 16s 0us/step


In [11]:
# checking images shape
X_train.shape, X_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [12]:
# display single image shape
X_train[0].shape

(32, 32, 3)

In [13]:
# checking labels
y_train[:5]

array([[6],
       [9],
       [9],
       [4],
       [1]], dtype=uint8)

In [14]:
# scaling image values between 0-1
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [15]:
# one hot encoding labels
y_train_encoded = keras.utils.to_categorical(y_train, num_classes = 10, dtype = 'float32')
y_test_encoded = keras.utils.to_categorical(y_test, num_classes = 10, dtype = 'float32')

In [16]:
def get_model():
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(32,32,3)),
        keras.layers.Dense(3000, activation='relu'),
        keras.layers.Dense(1000, activation='relu'),
        keras.layers.Dense(10, activation='sigmoid')    
    ])
    model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model

In [17]:
%%timeit -n1 -r1
# GPU
with tf.device('/GPU:0'):
    model_gpu = get_model()
    model_gpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 11s 6ms/step - loss: 1.8091 - accuracy: 0.3543
Epoch 2/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.6203 - accuracy: 0.4295
Epoch 3/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.5400 - accuracy: 0.4570
Epoch 4/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.4790 - accuracy: 0.4777
Epoch 5/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.4290 - accuracy: 0.4971
Epoch 6/10
1563/1563 [==============================] - 10s 6ms/step - loss: 1.3861 - accuracy: 0.5114
Epoch 7/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3479 - accuracy: 0.5258
Epoch 8/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.3132 - accuracy: 0.5405
Epoch 9/10
1563/1563 [==============================] - 10s 7ms/step - loss: 1.2828 - accuracy: 0.5502
Epoch 10/10
1563/1563 [==============================] - 10s 7ms/step - l

In [ ]:
%%timeit -n1 -r1
# CPU
with tf.device('/CPU:0'):
    model_cpu = get_model()
    model_cpu.fit(X_train_scaled, y_train_encoded, epochs = 10)

Epoch 1/10
1563/1563 [==============================] - 41s 26ms/step - loss: 1.8095 - accuracy: 0.3543
Epoch 2/10
1563/1563 [==============================] - 42s 27ms/step - loss: 1.6220 - accuracy: 0.4288
Epoch 3/10
1563/1563 [==============================] - 42s 27ms/step - loss: 1.5394 - accuracy: 0.4562
Epoch 4/10
1563/1563 [==============================] - 42s 27ms/step - loss: 1.4786 - accuracy: 0.4782
Epoch 5/10
1563/1563 [==============================] - 42s 27ms/step - loss: 1.4297 - accuracy: 0.4971
Epoch 6/10
1563/1563 [==============================] - 42s 27ms/step - loss: 1.3889 - accuracy: 0.5121
Epoch 7/10
 697/1563 [============>.................] - ETA: 22s - loss: 1.3488 - accuracy: 0.5244